In [9]:
import autosklearn.regression
from joblib import dump
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr, spearmanr
import sklearn
from skmisc.loess import loess
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score
import sys
sys.path.insert(0,'..')
from ml import *
import pandas as pd

### Preprocess data into right format for cider

In [2]:
feats = pd.read_csv('/data/togo_anon/feats/survey_combos/survey2018_cdr2018.csv')
feats = feats[[c for c in feats.columns if 'reporting' not in c or  c == 'reporting__number_of_records']]\
    .drop(['_c0', 'canton'], axis=1)\
    .rename({'phone_number':'name'}, axis=1)
feats.to_csv('/data/togo_anon/feats/survey_combos/survey2018_cdr2018_cider.csv', index=False)

/home/em/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
labels = pd.read_csv('/data/togo_anon/paper/datasets/survey2018.csv')
labels = labels[['phone_number', 'weight', 'cons']]\
    .rename({'phone_number':'name', 'cons':'label'}, axis=1)
labels.to_csv('/data/togo_anon/surveys/survey2018/survey2018_labels_cider.csv', index=False)

### Standard ML

In [16]:
learner = Learner(cfg_dir='../configs/config_emily.yml')
learner.merge()

Number of observations with features: 9421 (9421 unique)
Number of observations with labels: 8915 (8821 unique)
Number of matched observations: 8915 (8821 unique)


In [17]:
learner.tuned_model(model_name='gradientboosting')

/home/em/.conda/envs/cider/lib/python3.7/site-packages/joblib/numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/home/em/.conda/envs/cider/lib/python3.7/site-packages/joblib/numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


TypeError: load_model() got an unexpected keyword argument 'tuned'

In [ ]:
oos = learner.oos_predictions(model='gradientboosting', tuned=True)

In [ ]:
print('r2 score for gradient boosting: %.2f' % 
      r2_score(oos['true'], oos['predicted'], sample_weight=oos['weight']))

In [ ]:
learner.scatter_plot(model_name='gradientboosting', tuned=True)

### AutoML

In [18]:
learner.automl(model_name='autogluon')

/home/em/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:root:Presets specified: ['best_quality']
INFO:autogluon.tabular.learner.abstract_learner:Values in column 'weight' used as sample weights instead of predictive features. Evaluation will report weighted metrics, so ensure same column exists in test data.
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 3600s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "/home/em/cideroutputs/machinelearning/automl_models/autogluon/model/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.2.0
INFO:autogluon.tabular.learner.default_learner:Train Data Rows

INFO:autogluon.tabular.trainer.abstract_trainer:	4.62s	 = Validation runtime
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: CatBoost_BAG_L1 ... Training model for up to 1464.22s of the 2664.0s of remaining time.
ERROR:autogluon.tabular.trainer.abstract_trainer:	Warning: Exception caused CatBoost_BAG_L1 to fail during training (ImportError)... Skipping this model.
ERROR:autogluon.tabular.trainer.abstract_trainer:		`import catboost` failed.A quick tip is to install via `pip install catboost`.
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: ExtraTreesMSE_BAG_L1 ... Training model for up to 1464.15s of the 2663.93s of remaining time.
INFO:autogluon.tabular.trainer.abstract_trainer:	0.1218	 = Validation r2 score
INFO:autogluon.tabular.trainer.abstract_trainer:	14.12s	 = Training runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	4.9s	 = Validation runtime
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: XGBoost_BAG_L1 ... Training model for

INFO:autogluon.tabular.trainer.abstract_trainer:	0.1172	 = Validation r2 score
INFO:autogluon.tabular.trainer.abstract_trainer:	346.71s	 = Training runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	0.19s	 = Validation runtime
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 11.04s of the 10.96s of remaining time.
INFO:autogluon.tabular.trainer.abstract_trainer:	0.099	 = Validation r2 score
INFO:autogluon.tabular.trainer.abstract_trainer:	48.81s	 = Training runtime
INFO:autogluon.tabular.trainer.abstract_trainer:	4.65s	 = Validation runtime
INFO:autogluon.tabular.trainer.abstract_trainer:Completed 1/20 k-fold bagging repeats ...
INFO:autogluon.tabular.trainer.abstract_trainer:Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -44.79s of remaining time.
INFO:autogluon.tabular.trainer.abstract_trainer:	0.1282	 = Validation r2 score
INFO:autogluon.tabular.trainer.abstract_trainer:	0.4s	 = Tr

Finished automl training!


In [19]:
oos = learner.oos_predictions('autogluon', type='automl')

/home/em/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
print('R2 score for AutoML %.2f' % r2_score(oos['true'], oos['predicted'], sample_weight=oos['weight']))

R2 score for AutoML 0.14


In [21]:
learner.scatter_plot(model_name='automl', tuned=False)

/home/em/.local/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: scatter_plot() got an unexpected keyword argument 'tuned'